<a href="https://colab.research.google.com/github/adnan119/neural-network-architectures-in-pytorch/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **VGG16 in PyTorch**

In [1]:
import torch
import torch.nn as nn # neural networks modules
import torch.optim as optim # all optimization algorithms
import torch.nn.functional as F # functional module for non-parametric operations
import torchvision.datasets as datasets
import torchvision.transforms as transforms 
from torch.utils.data import DataLoader

In PyTorch you define your Models as subclasses of torch.nn.Module.

* In the __init__ function, you are supposed to initialize the layers you want to use. Unlike keras, Pytorch goes more low level and you have to specify the sizes of your network so that everything matches.

* In the forward method, you specify the connections of your layers. This means that you will use the layers you already initialized, in order to re-use the same layer for each forward pass of data you make.

* torch.nn.Functional contains some useful functions like activation functions a convolution operations you can use. However, these are not full layers so if you want to specify a layer of any kind you should use torch.nn.Module.

* You would use the torch.nn.Functional conv operations to define a custom layer for example with a convolution operation, but not to define a standard convolution layer.

In [2]:
# vgg architecture
VGG16 = [64,
         64,
         'MAX',
         128,
         128,
         'MAX',
         256,
         256,
         256,
         'MAX',
         512,
         512,
         512,
         'MAX',
         512,
         512,
         512,
         'MAX'] #Then flatten and 4096x4096x1000 add linear layers
         
# kernel_size = 3x3
# padding = 1
# stride = 1

In [7]:
class vgg16_net(nn.Module):
  
  def __init__(self, in_channels = 3, num_classes = 1000):
    super(vgg16_net,self).__init__() # <--- init of the parent method
    self.in_channels = in_channels
    self.conv_layers = self.create_conv_layers(VGG16)

    self.fcs = nn.Sequential(
        nn.Linear(512*7*7,4096),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(4096,4096),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(4096,num_classes)
    )

  def forward(self, x):
    x = self.conv_layers(x)
    x = x.reshape(x.shape[0],-1)
    x = self.fcs(x)
    return x

  def create_conv_layers(self, architecture):
    layers = []
    in_channels = self.in_channels

    for x in architecture:
      if type(x) == int:
        out_channels = x

        layers +=  [nn.Conv2d(in_channels = in_channels, out_channels = out_channels,
                              kernel_size = (3,3), padding = (1,1), stride = (1,1)),
                              nn.BatchNorm2d(x), #NOTE: batch-norm not included in the original implementation
                              nn.ReLU()]
        in_channels = x
      
      elif x == 'MAX':
        layers += [nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))]

    return nn.Sequential(*layers)

In [8]:
model = vgg16_net(in_channels=3, num_classes=1000)
x = torch.randn(1,3,224,224)
print(model(x).shape)

torch.Size([1, 1000])
